In the Vehicle detection part, there are 2 steps<br>
The first step : feature extraction<br>
The Second step : training processing<br>

In [ ]:
# This assignment has the following step
# 1st step: data processing(in the img_processing part)
# 2nd step: feature extraction
# 3rd step: training process
# 4th step: evaluation process

In [ ]:
# Here are some sample which implement the Vehicle detection based on HOG + Color histogram + SVM
# Source: https://github.com/ckirksey3/vehicle-detection-with-svm
# Another Chinese Version of Vehicle Detection
# Source: https://zhuanlan.zhihu.com/p/60229208

### Feature Extraction

In [ ]:
import numpy as np 
import cv2
import skimage.feature import hog

In [ ]:
# This Assignment used the Hog + color histogram
# This is Due to that the output vector can easily connect with each other
# You can also choose the SIFT, but SIFT needs to create the Bag of Words
# This will increate the work, also hard for freshman to implement

In [ ]:
# The Hog feature also used in object detection
# Source: https://www.analyticsvidhya.com/blog/2019/09/feature-engineering-images-introduction-hog-feature-descriptor/
# Example Of using Hog
# Source: https://scikit-image.org/docs/dev/auto_examples/features_detection/plot_hog.html

In [ ]:
# Here need to use Hog-----> Parameters
# Source: https://www.kite.com/python/docs/skimage.feature.hog#
# The return has 2 return which are newarr and hog_image
# You can also find this code from the following reference
# source: Source: https://github.com/ckirksey3/vehicle-detection-with-svm
def get_hog_features(imag, orient, pixel_per_cell, cell_per_block, visual=False, feature_vect=True):
    # visualise == False ----> hog img connot see
    if visual == False:
        features = hog(imag, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), block_norm= 'L2-Hys',
                       transform_sqrt=True, visualise=visual, feature_vector=feature_vect)
        return features
    elif visual == True:
        # visualise == True ----> hog img can be seen

        

### Training Process

### Evaluation Part